# Lab-05 Logistic Regrssion

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim

In [2]:
# 똑같은 결과를 재현해주기 위함
torch.manual_seed(1)

##### x_data = 공부시간 ex) x_data[0]은 1시간 강의듣고 2시간 lab session
##### y_data = 1:Pass, 2:Fail

In [9]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]] # 6*2
y_data = [[0], [0], [0], [1], [1], [1]] # 6*1

In [10]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

가설 계산

1/1+e^(-X*W+b)로도 표현할 수 있음

In [11]:
print(torch.exp(torch.FloatTensor([1])))

tensor([2.7183])


In [12]:
w = torch.zeros((2, 1), requires_grad=True) # d=2였기 때문에 행렬곱을 위해 2,1로 생성
b = torch.zeros(1, requires_grad=True)

In [13]:
h = 1 / (1+torch.exp(-(x_train.matmul(w)+b)))

In [14]:
print(h)
print(h.shape) # H(X)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


torch.sigmoid가 있어서 위의 과정이 필요없음

In [15]:
print(torch.sigmoid(torch.FloatTensor([1])))

tensor([0.7311])


In [16]:
h = torch.sigmoid(x_train.matmul(w) + b)

In [17]:
print(h)
print(h.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


hypothesis와 y_train간의 오차 측정 -> Cost Function

In [20]:
-(y_train[0] * torch.log(h[0]) + (1 - y_train[0]) * torch.log(1 - h[0]))

tensor([0.6931], grad_fn=<NegBackward>)

In [21]:
losses = -(y_train[0] * torch.log(h[0]) + (1 - y_train[0]) * torch.log(1 - h[0]))
print(losses)

tensor([0.6931], grad_fn=<NegBackward>)


In [22]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [24]:
f.binary_cross_entropy(h, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

#### Train

In [25]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [28]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [30]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = f.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [31]:
import numpy as np

### Real data 실습

In [ ]:
data = np.loadtxt('data-03-diabetes.csv', delimiter = ',', dtype = np.float32)
x_data = data[:, 0:-1]
y_data = data[:, [-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [ ]:
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    #cost = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean() 기존의 방법
    cost = f.binary_cross_entropy(hypothesis, y_train) # 토치쓰면 간단함

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

In [32]:
# 0.5를 기준으로 0인지 1인지 binary predictions
pred = hypothesis >= torch.FloatTensor([0.5])
pred[:5]

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])

In [33]:
y_train[:5]

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])

In [34]:
correct_pred = pred.float() == y_train
correct_pred[:5]

tensor([[True],
        [True],
        [True],
        [True],
        [True]])

In [35]:
acc = correct_pred.sum().item() / len(correct_pred)
print(acc*100)

100.0
